In [ ]:
#####code check from CP#################
!pip install -q torch torchvision transformers pydicom opencv-python Pillow accelerate
# Imports
import os, glob
import torch
import pydicom
import cv2
from PIL import Image
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer 


################GLOBAL PATHS
ROOT = Path("/content/drive/MyDrive")  
DICOM_DIR = ROOT / "PTXHeadtoHeadStudyData"
JPEG_DIR = Path("/content/cxr_jpegs")
JPEG_DIR.mkdir(exist_ok=True)



#####from the Hugging face page#################
import io
import requests
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer


# step 1: Setup constant
model_name = "StanfordAIMI/CheXagent-2-3b"
dtype = torch.bfloat16
device = "cuda"

# step 2: Load Processor and Model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
model = model.to(dtype)
model.eval()

# step 3: Inference
query = tokenizer.from_list_format([*[{'image': path} for path in paths], {'text': prompt}])
conv = [{"from": "system", "value": "You are a helpful assistant."}, {"from": "human", "value": query}]
input_ids = tokenizer.apply_chat_template(conv, add_generation_prompt=True, return_tensors="pt")
output = model.generate(
    input_ids.to(device), do_sample=False, num_beams=1, temperature=1., top_p=1., use_cache=True,
    max_new_tokens=512
)[0]
response = tokenizer.decode(output[input_ids.size(1):-1])
